In [ ]:
seoul_pm25 = pd.read_csv("./git/데이터/0. 프로토타입용 데이터/서울2023년초미세먼지데이터.csv")

In [ ]:
seoul_pm25.shape

In [ ]:
df.shape

In [ ]:
# 1월 3일 03시부터 시작하게 맞추기
seoul_pm25 = seoul_pm25.iloc[16:,:]

In [ ]:
seoul_pm25.head()

In [ ]:
seoul_pm25["일시"] = pd.to_datetime(seoul_pm25["일시"])

In [ ]:
seoul_pm25.head()

In [ ]:
df["일시"] = df["일시"].astype("datetime64")

In [ ]:
merged_pm25 = pd.merge(df, seoul_pm25, how = "inner", on = "일시")

In [ ]:
merged_pm25.isna().sum()

In [ ]:
x = merged_pm25.drop(["일시", "PM-10", "pm2.5"], axis = 1)
y = merged_pm25["pm2.5"]

In [ ]:
# 시퀀스 데이터 생성 함수
def create_sequences(x, y, time_steps=16):
    xs, ys = [], []
    for i in range(len(x) - time_steps):
        xs.append(x.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(xs), np.array(ys)

time_steps = 16
x_sequences, y_sequences = create_sequences(x, y, time_steps)

x_train, x_test, y_train, y_test = train_test_split(x_sequences, y_sequences, test_size=0.2, random_state=42)

In [ ]:
# LSTM 모델 정의
model = Sequential()
model.add(LSTM(units=256, return_sequences=True, input_shape=(time_steps, x_train.shape[2])))
model.add(keras.layers.Dropout(0.5))
model.add(LSTM(units=128))
model.add(keras.layers.Dropout(0.5))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 요약 출력
model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs = 200, validation_data= (x_val, y_val))